In [1]:
import os.path
import re
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import requests

from pandas.io.json import json_normalize


%matplotlib inline

import os
import sys

try:
    from urllib.parse import urlparse
except ImportError:
     from urlparse import urlparse

import seaborn as sns
import numpy as np

DATA_DIR = '../data/data_processed/' 

In [2]:
articles = pd.read_csv(DATA_DIR + 'NAS_articles_popularity_score.csv', index_col=0)
articles.index.names = ['id']

In [3]:
ordered_articles = articles.sort_values(by="popularity_score").reset_index()

In [4]:
ordered_articles = ordered_articles.reset_index()
ordered_articles = ordered_articles.rename(columns={'index': 'ranking'})
ordered_articles

,ranking,id,title,categories,scaled_length,scaled_views,scaled_refs,scaled_links,death,locations,end_date_clean,ongoing,popularity_score,article_lenght,views,refs_count,link_count
0,0,49532277,Hashimiya attack,civilian attack,0.000558,0.000021,0.000,0.000687,75,Hashimiya|Iraq,2014.0,False,0.001267,881,753,0,2
1,1,58421339,Fifth Third Center shooting,civilian attack,0.000431,0.000043,0.001,0.000687,0,NaN,0.0,False,0.002161,800,1501,1,2
2,2,49429205,January 2015 Maiduguri suicide bombing,civilian attack,0.000470,0.000011,0.001,0.002062,19,Maiduguri|Nigeria,0.0,False,0.003543,825,420,1,4
3,3,50567259,Transkei War,military conflict,0.000786,0.000019,0.000,0.002749,7000,South Africa Q258,1836.0,False,0.003554,1026,679,0,5
4,4,56640476,Operation of Sidi Aïch,military conflict,0.002870,0.000005,0.001,0.000000,0,Sidi Aïch|Tunisia,2015.0,False,0.003875,2351,206,1,1
5,5,32166814,Conquest of Ai,military conflict,0.000483,0.000021,0.000,0.003436,0,Ai Q403166,1400.0,False,0.003940,833,759,0,6
6,6,51033965,Punjab War,military conflict,0.000404,0.000205,0.000,0.003436,0,Punjab,1820.0,False,0.004046,783,6992,0,6
7,7,41345416,Battle of Deefa,military conflict,0.000566,0.000089,0.000,0.003436,0,Deefa|Oman,1968.0,False,0.004092,886,3056,0,6
8,8,3216047,Sack of Dun Gallimhe,military conflict,0.000182,0.000037,0.000,0.004124,0,NaN,1247.0,False,0.004343,642,1288,0,7
9,9,49532230,5 February 2014 Baghdad bombings,civilian attack,0.001437,0.000015,0.001,0.002062,32,Baghdad|Iraq,2014.0,False,0.004514,1440,542,1,4


### recupero dati dai results

In [5]:
r = requests.get('http://ada.ineeda.space/getResults.php')
print('Response body (parsed json):')
jsonResults = r.json()
results_pd = json_normalize(jsonResults)

Response body (parsed json):


In [6]:
results_pd[['ref1','ref2']]

,ref1,ref2
0,34181799,4764461
1,34276,34166492
2,53559919,52019122
3,13846188,32496189
4,43227423,49069111
5,12336420,52048842
6,24821786,19345883
7,201936,21493140
8,46230181,3832376
9,32964156,771


In [7]:
results_pd['ref1'] = results_pd.ref1.astype(int)
results_pd['ref2'] = results_pd.ref2.astype(int)
results_pd['result'] = results_pd.result.astype(int)

duplicati

In [8]:
a = results_pd.groupby(['ref1','ref2']).count()
a[a.result > 1]

result
ref1     ref2            
0        48545523       2
771      252846         2
         40950218       2
863      52717005       2
32611    57470685       2
32927    30207234       2
70181    140376         2
201936   51035921       2
799679   36141091       2
1021687  863            2
1059855  4764461        2
1439685  46215853       2
1652975  14049933       2
1856025  47894039       2
3030955  25892827       2
3257081  36494808       2
3770751  43227423       3
4006553  7746616        2
5043324  401909         2
         4478023        2
5137464  41181297       2
5357959  36197          2
6299613  46973028       5
7409287  36197          4
7746616  277783         2
7761623  32927          2
10396793 6730717        2
10852367 200987         9
11455647 22623199       2
12359069 32927          2
12697212 20855562       2
13425800 28160913       2
13735436 863            2
18423367 5058690        2
18842471 50792014       2
20683060 32611          2
22937234 32927          2
24417962 6750583        2
25566083 46230181       2
26397119 5043324        2
33800638 32611          2
34472007 51046467       2
42563890 6750583        2
43517689 43060786       2
44855655 41181297       3
45546772 863            2
48581121 799679         2
48958544 8442323        3
51076275 3171634        2
52019122 56704492       2
52048588 47894039       2
53415419 464071         2
58112781 4764461       13

In [9]:
results_pd[results_pd[['ref1','ref2']].duplicated(keep=False)]

,ref1,ref2,result
118,33800638,32611,2
119,33800638,32611,2
228,44855655,41181297,0
229,44855655,41181297,0
230,44855655,41181297,0
242,58112781,4764461,2
243,58112781,4764461,2
244,58112781,4764461,2
245,58112781,4764461,2
246,58112781,4764461,0


In [10]:
duplicated_res = results_pd[(results_pd.result != 0) & (results_pd[['ref1','ref2']].duplicated()) & ~(results_pd[['ref1','ref2','result']].duplicated())]
duplicated_res

,ref1,ref2,result
336,32611,57470685,1
881,34472007,51046467,2
1454,3030955,25892827,1
1533,0,48545523,2


ci sono 4 che sono duplicati e che si contraddicono.. sono 8 su 2000 e passa, io li tolgo

In [11]:
for index, row in duplicated_res.iterrows():
    results_pd = results_pd.drop(results_pd[(results_pd.ref1 == row.ref1) & (results_pd.ref2 == row.ref2)].index)

verifico che non ci siano più quei duplicati

In [12]:
results_pd[(results_pd.result != 0) & (results_pd[['ref1','ref2']].duplicated()) & ~(results_pd[['ref1','ref2','result']].duplicated())]

,ref1,ref2,result


perfetto!

## Inizio il ranking

In [13]:
ordered_articles[ordered_articles['id'] == 23497381].ranking.item()

12767

In [14]:
totalOk = 0

for index, row in results_pd[results_pd['result'] != 0].iterrows():
    articleID1 = row['ref1']
    articleID2 = row['ref2']
    result = row['result']
    
    ranking1 = ordered_articles[ordered_articles['id'] == articleID1].ranking.item()
    ranking2 = ordered_articles[ordered_articles['id'] == articleID2].ranking.item()
    
    predictedResult = (1, 2)[ranking1 < ranking2]
    
    if(predictedResult == result):
        totalOk+=1

In [15]:
totalResponses = len(results_pd[results_pd['result'] != 0].index)

In [16]:
percCorrects = totalOk/totalResponses*100

In [17]:
percCorrects

94.25612052730696

# Explain the questionaire

Finally we want to double check that our score is correlated with what people think. We writed a survey in order to verify that our metric was accurate.

We collected 2088 anserws from more than 20 people. The user choices between left, right or None in order to indicate the most popular conflict between the 2 proposed, i.e. it is a binary survey. 

The coverage of the survey is not increadible (12%) because we have more than 17 000 articles that can be used to write the survey, and some are very very unknown. Therefore, we randomly selected only articles that coresponded to the midle top part of our ranking.


We counted the number of correct answer for each popularity duel and we found out that 94 % of the answers to the questionaire matche our popularity score, not bad! 

The metric seems thus captured the popularity of the article. In this 6%, most of the errors seem to come more from a missclick or a confusion rather than an error in our metric. For instance, two users thought that World War I is less popular than 2009 Jaipur fire or Battle of Adwa, which seems a bit unrealistic. Another group of errors arise from 2 unknown conflicts, like 2008 Bin Salman mosque bombing vs the Battle of Marawi. 
Having established that the metric seems realistic we continue our investigation looking at the findings that the metric could give us.

In [18]:
results_pd.head()

,ref1,ref2,result
0,34181799,4764461,2
1,34276,34166492,2
2,53559919,52019122,2
3,13846188,32496189,1
4,43227423,49069111,1


In [19]:
articles = pd.read_csv('../data/data_processed/NAS_articles_popularity_score.csv', index_col=0)
articles.head()

,title,categories,scaled_length,scaled_views,scaled_refs,scaled_links,death,locations,end_date_clean,ongoing,popularity_score,article_lenght,views,refs_count,link_count
id,,,,,,,,,,,,,,,
771,American Revolutionary War,military conflict,0.375011,0.259929,0.598,0.634364,70000,Eastern North America|Caribbean Sea|Indian ...,1783.0,False,1.867305,238972,8794983,598,924
863,American Civil War,military conflict,0.319410,0.522999,0.212,0.532646,828000,Southern United States Q49042|Northern Uni...,1865.0,False,1.587055,203619,17696187,212,776
3793,Battle of Bosworth Field,military conflict,0.152325,0.032733,0.034,0.208935,0,Ambion Hill Q4741491|Battle of Bosworth F...,1485.0,False,0.427993,97380,1107588,34,305
4005,Battle of Pharsalus,military conflict,0.034510,0.017785,0.020,0.067354,230,Farsala Q985596,0.0,False,0.139650,22469,601822,20,99
4049,Battle of Blenheim,military conflict,0.104371,0.013543,0.095,0.137457,24968,Blindheim Q514914,1704.0,False,0.350371,66889,458284,95,201


In [20]:
results_pd['ref1'].drop_duplicates().values

array([34181799,    34276, 53559919, ..., 53756873, 32474888, 34963936])

#### Coverage

In [21]:
# coverage
distinct_ref = set(results_pd['ref1'].drop_duplicates().values) | set(results_pd['ref2'].drop_duplicates().values)
all_ref = set(articles.index.values)
len(all_ref - distinct_ref)/len(all_ref) * 100 # 1 - COVERAGE

88.32426238900028

#### find pop of our questions

In [22]:
results_pd.shape[0]

2080

In [23]:
# join with pop score our original date to find out the pop
ref1_pop = pd.DataFrame(index=results_pd['ref1'])
ref2_pop = pd.DataFrame(index=results_pd['ref2'])
ref1_pop = ref1_pop.join(articles[['popularity_score', 'title']])
ref2_pop = ref2_pop.join(articles[['popularity_score', 'title']])
ref1_pop.head()

,popularity_score,title
0,NaN,NaN
771,1.867305,American Revolutionary War
771,1.867305,American Revolutionary War
771,1.867305,American Revolutionary War
771,1.867305,American Revolutionary War


In [24]:
# join to result
results_pd = results_pd.join(ref1_pop, on='ref1')
results_pd.head()

,ref1,ref2,result,popularity_score,title
0,34181799,4764461,2,0.199419,Murder of Ann Ogilby
1,34276,34166492,2,1.518262,Yom Kippur War
1,34276,34166492,2,1.518262,Yom Kippur War
1,34276,34166492,2,1.518262,Yom Kippur War
1,34276,34166492,2,1.518262,Yom Kippur War


In [25]:
results_pd = results_pd.join(ref2_pop, on='ref2',  lsuffix='_ref1', rsuffix='_ref2')
results_pd.head()

,ref1,ref2,result,popularity_score_ref1,title_ref1,popularity_score_ref2,title_ref2
0,34181799,4764461,2,0.199419,Murder of Ann Ogilby,2.491002,World War I
0,34181799,4764461,2,0.199419,Murder of Ann Ogilby,2.491002,World War I
0,34181799,4764461,2,0.199419,Murder of Ann Ogilby,2.491002,World War I
0,34181799,4764461,2,0.199419,Murder of Ann Ogilby,2.491002,World War I
0,34181799,4764461,2,0.199419,Murder of Ann Ogilby,2.491002,World War I


In [26]:
results_pd = results_pd.drop_duplicates(subset=['ref1', 'ref2', 'result'])
print(results_pd.shape[0]) # we loose 80 anserwers that are duplicates

2008


In [27]:
# select conflicts that are misclassified
mis_conflicts_ref1 = results_pd[(results_pd.result != 0)\
                                & (results_pd.result == 2) \
                                & (results_pd.popularity_score_ref1 > results_pd.popularity_score_ref2)]


mis_conflicts_ref2 = results_pd[(results_pd.result != 0)\
                                & (results_pd.result == 1) \
                                & (results_pd.popularity_score_ref1 < results_pd.popularity_score_ref2)]

In [37]:
mis_conflicts_ref1

,ref1,ref2,result,popularity_score_ref1,title_ref1,popularity_score_ref2,title_ref2
1,34276,34166492,2,1.518262,Yom Kippur War,0.023883,Operación Lince Norte
140,34059,18467559,2,1.097418,War of 1812,0.018897,Battle of Lottorf
195,39120733,4566736,2,1.160729,Boston Marathon bombing,0.082398,East L.A. walkouts
266,51082294,30821764,2,1.057113,2016 Turkish coup d'état attempt,0.027580,Battle of Tadla
321,42563745,34566761,2,2.137529,War in Donbass,0.110393,1979 Herat uprising
327,4764461,5013,2,2.491002,World War I,0.309383,Battle of Adwa
339,42517963,40046379,2,0.780871,Migration to Xinjiang,0.016567,Battle of Casma
365,34059,53108932,2,1.097418,War of 1812,0.066654,Siege of Breslau (1757)
807,80197,33693442,2,1.260673,Soviet–Afghan War,0.019495,2007 West Azerbaijan clashes
827,6750583,446789,2,1.618953,History of Xinjiang,0.031322,Battle of Krefeld


In [38]:
mis_conflicts_ref2

,ref1,ref2,result,popularity_score_ref1,title_ref1,popularity_score_ref2,title_ref2
3,13846188,32496189,1,0.117815,Byzantine–Norman wars,1.138158,2011 Norway attacks
48,26121096,37930067,1,0.063930,Battle of Delaware Bay,1.107603,Sandy Hook Elementary School shooting
85,45625603,51082294,1,0.006558,Aden Airport attack,1.057113,2016 Turkish coup d'état attempt
96,6994817,863,1,0.055901,W. R. Myers High School shooting,1.587055,American Civil War
101,17248286,16772,1,0.021054,Siege of Danzig (1577),1.545874,Korean War
125,52166101,200987,1,0.098322,Mass killings of landlords under Mao Zedong,0.960319,Greek War of Independence
170,31860763,52019122,1,0.058460,Albania–Yugoslav border incident (April 1999),2.346496,Battle of Mosul (2016–2017)
260,46917969,34276,1,0.133203,Waterloo Campaign: Waterloo to Paris (2–7 July),1.518262,Yom Kippur War
268,17233989,54118340,1,0.015156,2008 Bin Salman mosque bombing,0.784813,Battle of Marawi
271,7668163,6750583,1,0.052011,Bombing of Duisburg in World War II,1.618953,History of Xinjiang


In [39]:
mis_all = set(mis_conflicts_ref1.title_ref1) | set(mis_conflicts_ref2.title_ref2)
len(mis_all) # sono più popolari di quello che dovevano essere

34

In [40]:
mis_all

{'1948 Arab–Israeli War',
 '2011 Norway attacks',
 '2012 Benghazi attack',
 '2014 pro-Russian unrest in Ukraine',
 "2016 Turkish coup d'état attempt",
 'American Civil War',
 'Appomattox Campaign',
 'Armenian Genocide',
 'Battle of Marawi',
 'Battle of Mosul (2016–2017)',
 'Boston Marathon bombing',
 'Euromaidan',
 'Greek War of Independence',
 'History of Xinjiang',
 'Iraq War',
 'Iraq War in Anbar Province',
 'Israeli–Syrian ceasefire line incidents during the Syrian Civil War',
 'Japanese invasions of Korea (1592–98)',
 'Kashmir conflict',
 'Korean War',
 'List of drone strikes in Pakistan',
 'Migration to Xinjiang',
 'Napoleonic Wars',
 'Revolutions of 1989',
 'Sandy Hook Elementary School shooting',
 'Saudi Arabian-led intervention in Yemen',
 'Soviet–Afghan War',
 'Springfield race riot of 1908',
 'Tiananmen Square protests of 1989',
 'War in Donbass',
 'War of 1812',
 'Winter War',
 'World War I',
 'Yom Kippur War'}

94 % of the answers to the questionaire matche our popularity score, not bad! 